**Following notebook contains the relevant code for Assignment 2, CS 565 - Intelligent Systems and Interfaces**

# Prequisites

In [ ]:
# Mounting the google drive to directly access the dataset.
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# NLTK will be used for word tokenization and sentence segmentation.
!pip install inltk

In [ ]:
import nltk
nltk.download('punkt')

import numpy as np
import random
import math

#Dataset

In [ ]:
# The address has to change according to the location of file on the drive.
path = '/content/drive/My Drive/Data/en_wiki.txt'
en_wiki = open(path,'r').read()

# Using NLTK Sentence Tokenizer
dataset = nltk.sent_tokenize(en_wiki)

print(len(dataset))
print(dataset[0 : 5])

761582
['The word "atom" was coined by ancient Greek philosophers.', 'However, these ideas were founded in philosophical and theological reasoning rather than evidence and experimentation.', 'As a result, their views on what atoms look like and how they behave were incorrect.', 'They also could not convince everybody, so atomism was but one of a number of competing theories on the nature of matter.', 'It was not until the 19th century that the idea was embraced and refined by scientists, when the blossoming science of chemistry produced discoveries that only the concept of atoms could explain.']


In [ ]:
# Shuffling the sentences in the dataset
random.shuffle(dataset)
print(dataset[0 : 5])

# Splitting the dataset into training and testing data
train_data = dataset[0 : round(0.9*len(dataset))]
test_data = dataset[round(0.9*len(dataset)) : ]

print(len(train_data), len(test_data))
del dataset

['Poets increasingly developed a self-conscious relationship to tradition, which took the form of a new emphasis on craftsmanship of expression and an idiosyncratic freedom in allusions to Classical and Biblical sources.', 'Dupin puts an advertisement in the newspaper asking if anyone has lost an "Ourang-Outang".', 'Lilies are usually planted as bulbs in the dormant season.', 'There is considerable commercial interest in the field because of its application to news-gathering, text categorization, voice-activation, archiving, and large-scale content-analysis.', 'He appeared again as Poirot in three made-for-television movies: "Thirteen at Dinner" (1985), "Dead Man\'s Folly" (1986), and "Murder in Three Acts" (1986).']
685424 76158


# N-Gram Language Model

In [ ]:
# Creating 5 sets of training and development data
train_data_list = []
development_data_list = []

for i in range(5):
  random.shuffle(train_data)
  train_data_list.append(train_data[0 : round(0.9*len(train_data))])
  development_data_list.append(train_data[round(0.9*len(train_data)) : ])

print(len(train_data_list[0]))
print(len(development_data_list[0]))

del train_data

616882
68542


In [ ]:
# Finding out counts and list of unigram, bigrams and trigrams.
# Note: The threshold is used to group very rare words into a seperate UNKNOWN category
def ngram_train(sentences, threshold):
  list_of_bigrams = []
  list_of_trigrams = []

  unigram_count = {}
  bigram_count = {}
  trigram_count = {}

  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words)):
      
      # If the word can be at the beginning of a bigram
      if i < len(words)-1:
        list_of_bigrams.append((words[i], words[i + 1]))
        if (words[i], words[i + 1]) in bigram_count:
          bigram_count[(words[i], words[i + 1])] += 1
        else:
          bigram_count[(words[i], words[i + 1])] = 1

      # If the word can be at the beginning of a trigram
      if i < len(words)-2:
        list_of_trigrams.append((words[i], words[i + 1], words[i + 2]))
        if (words[i], words[i + 1], words[i + 2]) in trigram_count:
          trigram_count[(words[i], words[i + 1], words[i + 2])] += 1
        else:
          trigram_count[(words[i], words[i + 1], words[i + 2])] = 1
      
      if words[i] in unigram_count:
        unigram_count[words[i]] += 1
      else:
        unigram_count[words[i]] = 1

  unigram_count["unknown"] = 0
  for unigram in unigram_count:
    if (unigram_count[unigram] < threshold):
      unigram_count["unknown"] += 1
      unigram_count[unigram] = 0

  return list_of_bigrams, list_of_trigrams, unigram_count, bigram_count, trigram_count

**Note that the below steps can be repeated for all the training data sets created by changing the index to 0, 1, 2, 3 and 4.**

In [ ]:
# Calling ngram_train to find out the list and count of ngrams
threshold = 4
list_of_bigrams, list_of_trigrams, unigram_count, bigram_count, trigram_count = ngram_train(train_data_list[0], threshold)

print(len(list_of_bigrams))
print(list_of_bigrams[0:5])

print(len(list_of_trigrams))
print(list_of_trigrams[0:5])

12947650
[('The', 'inward'), ('inward', 'code'), ('code', 'assists'), ('assists', 'in'), ('in', 'the')]
12332527
[('The', 'inward', 'code'), ('inward', 'code', 'assists'), ('code', 'assists', 'in'), ('assists', 'in', 'the'), ('in', 'the', 'delivery')]


In [ ]:
# Assigns onegram probabilties to each element
def calculate_onegram_prob(unigram_count):
  onegram_probability = {}
  total_onegram_count = 0

  for onegram in unigram_count:
    total_onegram_count += unigram_count[onegram]

  for onegram in unigram_count:
    onegram_probability[onegram] = unigram_count[onegram] / total_onegram_count

  return onegram_probability

In [ ]:
# Assigns bigram probabilties to each element
def calculate_bigram_prob(list_of_bigrams, unigram_count, bigram_count):
  bigram_probability = {}

  for bigram in list_of_bigrams:
    if unigram_count.get(bigram[0]) == 0:
      bigram_probability[bigram] = 0
    else:
      bigram_probability[bigram] = bigram_count[bigram] / (unigram_count.get(bigram[0]))

  return bigram_probability


In [ ]:
# Assigns bigram probabilties to each element
def calculate_trigram_prob(list_of_trigrams, bigram_count, trigram_count):
  trigram_probability = {}

  for trigram in list_of_trigrams:
    if bigram_count.get((trigram[0], trigram[1])) == 0:
      trigram_probability[trigram] = 0
    else:
      trigram_probability[trigram] = trigram_count[trigram] / (bigram_count.get((trigram[0], trigram[1])))

  return trigram_probability

In [ ]:
# Pre-computing probabilities for further sections
onegram_prob = calculate_onegram_prob(unigram_count)
bigram_prob  = calculate_bigram_prob(list_of_bigrams, unigram_count, bigram_count)
trigram_prob = calculate_trigram_prob(list_of_trigrams, bigram_count, trigram_count)

In [ ]:
print(onegram_prob['the'])
print(bigram_prob[('of', 'the')])
print(trigram_prob[('value', 'of', 'the')])

0.06507428053881525
0.27801736915282826
0.3549382716049383


## Interpolation Smoothing

In [ ]:
# Gets the counts of all the trigrams present in the test set
def get_trigram_count(sentences):
  trigram_count = {}
  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words) - 2):
      if (words[i], words[i + 1], words[i + 2]) in trigram_count:
        trigram_count[(words[i], words[i + 1], words[i + 2])] += 1
      else:
        trigram_count[(words[i], words[i + 1], words[i + 2])] = 1
  return trigram_count

In [ ]:
# Returns the interpolated perplexity of the given list of sentences.
def calculate_interpolation_lambda(sentences, test_trigram_count, onegram_prob, bigram_prob, trigram_prob, lambda_set):
  likelihood = 0
  
  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words) - 2):
      trigram = (words[i], words[i + 1], words[i + 2])
      if trigram in trigram_prob:
        tri_prob = trigram_prob[trigram]
      else:
        tri_prob = 0

      bigram = (words[i], words[i + 1])
      if bigram in bigram_prob:
        bi_prob = bigram_prob[bigram]
      else:
        bi_prob = 0

      onegram = words[i]
      if onegram in onegram_prob:
        one_prob = onegram_prob[onegram]
      else:
        one_prob = 0

      # print(one_prob, bi_prob, tri_prob)
      prob = lambda_set[0]*one_prob + lambda_set[1]*bi_prob + lambda_set[2]*tri_prob
      
      if prob != 0:
        # print(math.log(prob, 2))
        likelihood += np.log2(prob)*test_trigram_count[trigram]

  return likelihood

In [ ]:
# Returns the interpolated perplexity of the given list of sentences.
def calculate_interpolation_perplexity(sentences, test_trigram_count, onegram_prob, bigram_prob, trigram_prob, lambda_set):
  count = 0
  likelihood = 0
  
  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words) - 2):
      trigram = (words[i], words[i + 1], words[i + 2])
      if trigram in trigram_prob:
        tri_prob = trigram_prob[trigram]
      else:
        tri_prob = 0

      bigram = (words[i], words[i + 1])
      if bigram in bigram_prob:
        bi_prob = bigram_prob[bigram]
      else:
        bi_prob = 0

      onegram = words[i]
      if onegram in onegram_prob:
        one_prob = onegram_prob[onegram]
      else:
        one_prob = 0

      # print(one_prob, bi_prob, tri_prob)
      prob = lambda_set[0]*one_prob + lambda_set[1]*bi_prob + lambda_set[2]*tri_prob
      
      if prob != 0:
        # print(math.log(prob, 2))
        
        likelihood += math.log(prob, 2)*test_trigram_count[trigram]
      count += 1

  perplexity = math.pow(2, -1*(likelihood/count))
  return perplexity

**Note that the below steps can be repeated for all the corresponding development data sets created by changing the index to 0, 1, 2, 3 and 4.**

In [ ]:
test_trigram_count = get_trigram_count(development_data_list[0])

max_lambda = []
max_likelihood = float('-inf')

# Iterate over lambda values to find the ideal parameters 
for lambda1 in range(1, 10, 1):
  for lambda2 in range(1, 10-lambda1, 1):
    lambda_set = [lambda1/10, lambda2/10, (10 - lambda1 - lambda2)/10]
    likelihood = calculate_interpolation_lambda(development_data_list[0], test_trigram_count, onegram_prob, bigram_prob, trigram_prob, lambda_set)
    
    if (likelihood > max_likelihood):
      max_lambda = lambda_set
      max_likelihood = likelihood

print(max_lambda, max_likelihood)

perplexity = calculate_interpolation_perplexity(development_data_list[0], test_trigram_count, onegram_prob, bigram_prob, trigram_prob, max_lambda)
print(perplexity)

[0.8, 0.1, 0.1] -0.063324085910393
1.043956568032204


In [ ]:
test_trigram_count = get_trigram_count(test_data)

perplexity = calculate_interpolation_perplexity(test_data, test_trigram_count, onegram_prob, bigram_prob, trigram_prob, max_lambda)
print(perplexity)

1.028583701239205


In [ ]:
# In case RAM isnt sufficient
del test_trigram_count

## Laplace Smoothing

In [ ]:
# Gets the counts of all the trigrams present in the test set
def get_trigram_count(sentences):
  trigram_count = {}
  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words) - 2):
      if (words[i], words[i + 1], words[i + 2]) in trigram_count:
        trigram_count[(words[i], words[i + 1], words[i + 2])] += 1
      else:
        trigram_count[(words[i], words[i + 1], words[i + 2])] = 1
  return trigram_count

In [ ]:
# Assigns onegram probabilties with laplace smoothing, we can change the value of k. 
def calculate_laplace_smoothing(list_of_trigrams, trigram_count):
  k = 1
  trigram_laplace_probability = {}
  
  for trigram in list_of_trigrams:
    trigram_laplace_probability[trigram] = (trigram_count[trigram]+k) / (bigram_count.get((trigram[0], trigram[1])) + k*len(list_of_trigrams))

  return trigram_laplace_probability

In [ ]:
trigram_laplace_prob = calculate_laplace_smoothing(list_of_trigrams, trigram_count)
print(trigram_laplace_prob[('He', 'is', 'the')])

8.513323919488928e-06


In [ ]:
def calculate_laplace_perplexity(test_trigram_count, trigram_laplace_prob):
  likelihood = 0
  for trigram in test_trigram_count:
    if trigram in trigram_laplace_prob:
      likelihood += test_trigram_count[trigram] * np.log2(trigram_laplace_prob[trigram])

  count = 0
  for trigram in test_trigram_count:
    count += test_trigram_count[trigram]
  
  likelihood = likelihood / count
  return pow(2,-1*likelihood), likelihood

**Note that the below steps can be repeated for all the corresponding development data sets created by changing the index to 0, 1, 2, 3 and 4.**

In [ ]:
test_trigram_count = get_trigram_count(development_data_list[0])

perplexity, likelihood = calculate_laplace_perplexity(test_trigram_count, trigram_laplace_prob)
print(perplexity, likelihood)

177.01362939772972 -7.467716636566069


In [ ]:
test_trigram_count = get_trigram_count(test_data)

perplexity, likelihood = calculate_laplace_perplexity(test_trigram_count, trigram_laplace_prob)
print(perplexity, likelihood)

179.26432942256946 -7.485944634563072


In [ ]:
# In case RAM isnt sufficient
del trigram_laplace_prob

## Discount Smoothing

In [ ]:
# Gets the counts of all the unigrams present in the test set
def get_unigram_count(sentences):
  unigram_count = {}
  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words)):
      if words[i] in unigram_count:
        unigram_count[words[i]] += 1
      else:
        unigram_count[words[i]] = 1
  return unigram_count

In [ ]:
# Gets the counts of all the bigrams present in the test set
def get_bigram_count(sentences):
  bigram_count = {}
  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words) - 1):
      if (words[i], words[i + 1]) in bigram_count:
        bigram_count[(words[i], words[i + 1])] += 1
      else:
        bigram_count[(words[i], words[i + 1])] = 1
  return bigram_count

In [ ]:
# Gets the counts of all the trigrams present in the test set
def get_trigram_count(sentences):
  trigram_count = {}
  for sentence in sentences:
    words = sentence.split()
    for i in range(len(words) - 2):
      if (words[i], words[i + 1], words[i + 2]) in trigram_count:
        trigram_count[(words[i], words[i + 1], words[i + 2])] += 1
      else:
        trigram_count[(words[i], words[i + 1], words[i + 2])] = 1
  return trigram_count

In [ ]:
# Finding out probabilities corresponding to a particular value of beta
def calculate_bigram_discounting_beta(unigram_count, bigram_count, beta, unigram_prob):
  current_prob = {}

  for i in unigram_count:
    sum_prob = 0
    sigms = 0

    for j in unigram_count:
      if bigram_count[(i, j)] != 0:
        current_prob[(i, j)] = (bigram_count[(i, j)] - beta)/ unigram_count[i]
        sum_prob += current_prob[(i, j)]
      else:
        sigms += unigram_prob[j]

    alpha = 1 - sum_prob
    for j in unigram_count:
      if bigram_count[(i, j)] == 0:
        current_prob[(i, j)] = alpha*(unigram_prob[j] / sigms)

  return current_prob

In [ ]:
# Find out the parameter and implement discounting smoothing for bigrams.
def calculate_bigram_discounting(unigram_count, bigram_count, unigram_prob, test_bigram_count):
  bigram_probability = {}
  max_likelihood = float('-inf')
  max_beta = 0

  for beta in range(10):
    beta = beta/10
    likelihood = 0
    current_prob = calculate_bigram_discounting_beta(unigram_count, bigram_count, beta, unigram_prob)

    for bigram in test_bigram_count:
      likelihood += test_bigram_count[bigram]*np.log2(current_prob[bigram])

    if likelihood > max_likelihood:
      max_beta = beta
      max_likelihood = likelihood
      bigram_probability = current_prob

  return max_beta, bigram_probability


In [ ]:
# Finding out probabilities corresponding to a particular value of beta
def calculate_trigram_discounting_beta(unigram_count, bigram_count, trigram_count, beta, bigram_prob):
  current_prob = {}

  for i in unigram_count:
    for j in unigram_count:
      sum_prob = 0
      sigms = 0

      for k in unigram_count:
        trigram = (i, j, k)

        if trigram_count[trigram] != 0:
          current_prob[trigram] = (trigram_count[trigram] - beta)/ bigram_count[(i, j)]
          sum_prob += current_prob[trigram]
        else:
          sigms += bigram_prob[(j, k)]

      alpha = 1 - sum_prob
      for k in unigram_count:
        if trigram_count[trigram] == 0:
          current_prob[trigram] = alpha*(bigram_prob[(j, k)] / sigms)
  
  return current_prob

In [ ]:
# Find out the parameter and implement discounting smoothing for trigrams.
def calculate_trigram_discounting(unigram_count, bigram_count, trigram_count, bigram_prob, test_trigram_count):
  trigram_probability = {}
  max_likelihood = float('-inf')
  max_beta = 0

  for beta in range(10):
    beta = beta/10
    likelihood = 0
    current_prob = calculate_trigram_discounting_beta(unigram_count, bigram_count, trigram_count, beta, bigram_prob)

    for trigram in test_trigram_count:
      likelihood += test_trigram_count[trigram]*np.log2(current_prob[trigram])

    if likelihood > max_likelihood:
      max_beta = beta
      max_likelihood = likelihood
      trigram_probability = current_prob

  return max_beta, trigram_probability


In [ ]:
def calculate_discounting_perplexity(test_unigram_count, test_trigram_count, trigram_prob):
  likelihood = 0
  for trigram in test_trigram_count:
    count = test_trigram_count[trigram]
    if trigram_prob[trigram] != 0:
      likelihood += count*np.log2(trigram_prob[trigram])
  
  num = 0
  for unigram in test_unigram_count:
    num += test_unigram_count[unigram]
  likelihood = likelihood / num

  return pow(2,-1*likelihood), likelihood

**Note that the below steps can be repeated for all the corresponding development data sets created by changing the index to 0, 1, 2, 3 and 4.**

In [ ]:
test_unigram_count = get_unigram_count(development_data_list[0])
test_bigram_count = get_bigram_count(development_data_list[0])
test_trigram_count = get_trigram_count(development_data_list[0])

beta_bigram, bigram_prob = calculate_bigram_discounting(unigram_count, bigram_count, onegram_prob, test_bigram_count)
beta_trigram, trigram_prob = calculate_trigram_discounting(unigram_count, bigram_count, trigram_count, bigram_prob, test_trigram_count)

perplexity, likelihood = calculate_discounting_perplexity(test_unigram_count, test_trigram_count, trigram_prob)
print(perplexity, likelihood)

11.85167084255618 -3.566921492304502
